In [ ]:
# pip install pandas
# pip install matplotlib
# pip install seaborn
# pip install latex
# pip install beautifulsoup4
# pip install sklearn
# pip install numpy
# pip install urlextract
# pip install wordcloud
# pip install scikit-plot
# pip install transformers
# pip install tensorflow
# pip install lxml
# pip list
# pip install --upgrade pip


In [ ]:
import pandas as pd

df = pd.read_csv("fraud_email_.csv")
pd.set_option('display.max_colwidth', 500)
df["id"] = df.index

first_column = df.pop('id')
df.insert(0, 'id', first_column)
df.columns = [x.lower() for x in df.columns] #lowercase headers

df.head(5)
# class == 1 is fraud email

In [ ]:
# add test row
df.loc[-1] = [-1, 'http://www.google.com http:/ew.bbc.com http://www.facebook.com yomama@gmail.com fakeass@huh', 1]  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 
df.head()

In [ ]:
#Converting the text to lower case 
#import re
#df['Text'] = df['Text'].astype(str).apply(lambda x: x.lower()) 
#df.head(5)

In [ ]:
#Remove "=" symbol from data
df['text'] = df['text'].astype(str) # cast as string
df['text'] = df['text'].apply(lambda x: x.replace("=",''))
df.head(5)

In [ ]:
df.iloc[0,1:2]['text']

In [ ]:
df.describe()

In [ ]:
from urlextract import URLExtract

#extractor = URLExtract()
#df1 = df.copy()
#df1.Text = df1.Text.apply(extractor.find_urls)
#df1.head()

In [ ]:
df2 = df.copy()

import re

# ** ASSUMPTION THAT url be it fake or real, starts with http. if there is fake url that do not start with http,
# it is not detected as a url.

URLPATTERN = r'(HTTP:|http\S+)'
# urlpattern2 = '(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+' --> cant find a best regex to identify url

EMAILPATTERN = r"[a-zA-Z0-9\.\-+_]+@[a-zA-Z0-9\.\-+_]+[a-zA-Z0-9\.\-+_]+"

df2['urlcount'] = df2.text.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
df2['emailcount'] = df2.text.apply(lambda x: re.findall(EMAILPATTERN, x)).str.len()
df2['url'] = df2.text.apply(lambda x: re.findall(URLPATTERN, x))
df2['email'] = df2.text.apply(lambda x: re.findall(EMAILPATTERN, x))

#df2['urls'] = ([int(s) for s in list_nombre.split(' ')]
    #[for i in range(d df2.Text.apply(lambda x: re.findall(URLPATTERN, x)).str
#df2.groupby('id').sum()['urlcount']
df2.head()

In [ ]:
df4 = df2.copy()
df4.dtypes

In [ ]:
df5 = df4.loc[(df4['urlcount'] > 0) & (df4['emailcount'] > 0)]
df5.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
pd.crosstab(df5['class'],df5.urlcount).plot(kind='bar')
plt.title('Fraud Emails')
plt.xlabel('Frequency of URLs in Emails')
plt.ylabel('Frequency of URLs')

pd.crosstab(df5['class'],df5.emailcount).plot(kind='bar')
plt.title('Fraud Emails')
plt.xlabel('Frequency of Emails in Emails')
plt.ylabel('Frequency of Emails')


In [ ]:
urldf = df2[['id', 'url', 'class']]
emaildf = df2[['id', 'email', 'class']]
emaildf.head(10)

In [ ]:
# version 2 of url and email df only contain rows with email/url data coz dh == unable to understand classification

urldf2 = urldf[urldf['url'].map(lambda d: len(d)) > 0]
emaildf2 = emaildf[emaildf['email'].map(lambda d: len(d)) > 0]
emaildf2.head()

In [ ]:
# generate clean and fraud list of emails/url | for own purposes
fraudemaillist = []
fraudurllist = []

def coltolist(dataf, col):
    fraudlist = []
    cleanlist = []
    
    for index, row in dataf.iterrows():
        if row['class'] == 1:
            for rowdata in row[col]:
                fraudlist.append(rowdata)
        else:
            for rowdata in row[col]:
                cleanlist.append(rowdata)
    return [fraudlist, cleanlist]

keke = [urldf2, emaildf2]
keke2 = ['url', 'email']
tgt = []

for i in range(len(keke)):
    tgt.append(coltolist(keke[i], keke2[i]))

fraudurl = tgt[0][0]
cleanurl = tgt[0][1]
fraudemail = tgt[1][0]
cleanemail = tgt[1][1]

cleanemail[:5]

In [ ]:
# urldf2 , emaildf2 without ids
urldf3 = urldf2.drop(['id'], axis=1)
emaildf3 = emaildf2.drop(['id'], axis=1)
emaildf3

In [ ]:
fraudurldf = pd.DataFrame(fraudurl, columns =['url'])
fraudurldf['class']=1
cleanurldf = pd.DataFrame(cleanurl, columns =['url'])
cleanurldf['class']=0
fraudemaildf = pd.DataFrame(fraudemail, columns =['email'])
fraudemaildf['class']=1
cleanemaildf = pd.DataFrame(cleanemail, columns =['email'])
cleanemaildf['class']=0
cleanurldf

In [ ]:
urls = [fraudurldf, cleanurldf]
url = pd.concat(urls)
emails = [fraudemaildf, cleanemaildf]
email = pd.concat(emails)
email
# ************************************ now url and email df contain each email (1 per row) with associated class

In [ ]:
df3 = df2.copy()
df3 = df3[['text', 'class']]
df3

In [ ]:
df3['class'].value_counts()

In [ ]:
import seaborn as sns 
import matplotlib.pyplot  as plt
sns.countplot(x='class', data = df3, palette = 'Pastel1')
plt.show()
plt.savefig('count_plot')

# Naive Bayes Classifier

## I didn't clean the data by stopwords, punctuation, lowercase, punctuation removals, bad characters etc because tbh I feel fraud messages usually are phrased really badly and thats their characteristic.

In [ ]:
df3.head()

In [ ]:
from bs4 import BeautifulSoup

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    return text
    
df3['text'] = df3['text'].apply(clean_text)
df3['text'] = df3['text'].str.replace('_',' ')
df3.loc[[700]]

In [ ]:
from sklearn.model_selection import train_test_split

X = df3.text
y = df3['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy: %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

# Understanding Term Distribution

In [ ]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = []
for x,y in df3.iterrows():
    corpus.append(y['text'])

vectorizer = TfidfVectorizer()
data = vectorizer.fit_transform(corpus)
names = vectorizer.get_feature_names()

# http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

cvec = CountVectorizer(min_df=.0025, max_df=.1)
cvec.fit(df.text)
cvec_counts = cvec.transform(df.text)

occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by=['occurrences'], ascending = False)

In [ ]:
frauddf = df[df['class'] == 1]
cvec = CountVectorizer(min_df=.0025, max_df=.1)
cvec.fit(frauddf.text)
cvec_counts = cvec.transform(frauddf.text)
fraudterms = cvec.get_feature_names

occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
fraudcounts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences_in_fraud': occ})
fraudcounts_df.sort_values(by=['occurrences_in_fraud'], ascending = False)

In [ ]:
result = pd.merge(counts_df,fraudcounts_df,on='term', how='left')
resultv2 = result.fillna(0)
result

In [ ]:
resultv3 = resultv2.copy()
# resultv2 takes into consideration NaN values into 0.
resultv2

In [ ]:
resultv2['occurrences_in_non_fraud'] = resultv2['occurrences'] - resultv2['occurrences_in_fraud']
resultv2["word_in_fraud_percentage"] = resultv2["occurrences_in_fraud"] / resultv2['occurrences'] * 100
resultv2

In [ ]:
resultv2['word_in_non_fraud_percentage'] = 100 - resultv2['word_in_fraud_percentage']
resultv2

In [ ]:
# standardized
# rationale is beacuse some have more occurrence in fraud resulting in higher chance of getting fraud % higher
# so we need to standardize to normalize them to similar standings

resultv3['occurrences'] = np.log(resultv3['occurrences'])
resultv3['occurrences_in_fraud'] = np.log(resultv3['occurrences_in_fraud'])

resultv4 = resultv3.fillna(0)
resultv4["word_in_fraud_percentage"] = resultv4["occurrences_in_fraud"] / resultv4['occurrences'] * 100
resultv4 = resultv4.fillna(0)
resultv4

# tbh i lost my thoughts on why i rationalized but the data got some -inf

In [ ]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ, 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(8)

In [ ]:
resultv2[['term', 'word_in_fraud_percentage']].sort_values(by='word_in_fraud_percentage', ascending = False)

In [ ]:
resultv2['occurrences_in_fraud'] = resultv2['occurrences_in_fraud'].apply(np.int64)
resultv2['occurrences_in_non_fraud'] = resultv2['occurrences_in_non_fraud'].apply(np.int64)
resultv2

In [ ]:
# Too long to run
# plt.figure(figsize=(15,10))
# resultv2.sort_values(by="word_in_fraud_percentage",ascending=False)["word_in_fraud_percentage"].plot.bar()
# plt.xticks(rotation=50)
# plt.xlabel("Term")
# plt.ylabel("Percentage of Term in Fraud Emails")
# plt.show()

# Understanding Term Distribution - WordCloud

In [ ]:
fraudcorpus = ''
nonfraudcorpus = ''

for index, row in resultv2.iterrows():
    for i in range(row['occurrences_in_fraud']):
        fraudcorpus += row['term'] + ' '
    for i in range(row['occurrences_in_non_fraud']):
        nonfraudcorpus += row['term'] + ' '

In [ ]:
nonfraudcorpus

In [ ]:
from wordcloud import WordCloud

# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=300, max_words=100, background_color="white", collocations=False).generate(fraudcorpus)

# Display the generated image:
plt.figure(figsize=(14,9))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud.to_file("img/fraudword_wordcloud_with_stopwords.png")

In [ ]:
from wordcloud import WordCloud

# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=300, max_words=100, background_color="white", collocations=False).generate(nonfraudcorpus)

# Display the generated image:
plt.figure(figsize=(14,9))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# wordcloud.to_file("img/non_fraudword_wordcloud_with_stopwords.png")

In [ ]:
from wordcloud import WordCloud

# Create and generate a word cloud image:
wordcloud = WordCloud(stopwords = 'english', max_font_size=50, max_words=100, background_color="white", collocations=False).generate(fraudcorpus)

# Display the generated image:
plt.figure(figsize=(14,9))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud.to_file("img/fraudword_wordcloud_without_stopwords.png")

In [ ]:
from wordcloud import WordCloud

# Create and generate a word cloud image:
wordcloud = WordCloud(stopwords = 'english', max_font_size=50, max_words=100, background_color="white", collocations=False).generate(nonfraudcorpus)

# Display the generated image:
plt.figure(figsize=(14,9))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud.to_file("img/non_fraudword_wordcloud_without_stopwords.png")

In [ ]:
# words = cvec.get_feature_names()
# fraud_dic = {words[x]:int(0) for x in range(len(words))}
# #list(fraud_dic.items())[:4]
# fraud_dic

In [ ]:
counts_df

In [ ]:
np.asarray(cvec_counts.sum(axis=0))

In [ ]:
cvec.get_feature_names()

In [ ]:
# newname = [x for x in names if not any(c.isdigit() for c in x)]

# for x,y in counts_df.iterrows():
#     for ch in y.term:
#         if any(ch.isdigit()):
            

# digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# counts_df[~counts_df.term.isin(digits)]
# counts_df

# Linear Support Vector Machine

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print('accuracy: %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# strings to raw docs with tfidf features to floats
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=1000)),
               ])

logreg.fit(X_train, y_train)
fit = logreg.fit(X_train, y_train)

coef = logreg.named_steps['clf'].coef_[0]

y_pred = logreg.predict(X_test)

print('accuracy: %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

In [ ]:
coef

In [ ]:
# https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# import sklearn.linear_model
# plt.rc('text', usetex=True)

# # Retrieve the model parameters.
# print(logreg.named_steps['clf'].coef_[0][:2])
# w1, w2 = logreg.named_steps['clf'].coef_[0][:2]
# # Calculate the intercept and gradient of the decision boundary.
# c = -b/w2
# m = -w1/w2

# # Plot the data and the classification with the decision boundary.
# xmin, xmax = -1, 2
# ymin, ymax = -1, 2.5
# xd = np.array([xmin, xmax])
# yd = m*xd + c
# plt.plot(xd, yd, 'k', lw=1, ls='--')
# plt.fill_between(xd, yd, ymin, color='tab:blue', alpha=0.2)
# plt.fill_between(xd, yd, ymax, color='tab:orange', alpha=0.2)

# plt.scatter(*X_train[y_train==0].T, s=8, alpha=0.5)
# plt.scatter(*X_train[y_train==1].T, s=8, alpha=0.5)
# plt.xlim(xmin, xmax)
# plt.ylim(ymin, ymax)
# plt.ylabel(r'$x_2$')
# plt.xlabel(r'$x_1$')

# plt.show()

In [ ]:
#cdf = pd.DataFrame(LogisticRegression.coef_, X.columns, columns=['Coefficients'])
#print(cdf)

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

# Sentimental Analysis

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from scikitplot.metrics import plot_confusion_matrix

In [ ]:
df4.head()

In [ ]:
cleaned = df4.copy()
    
cleaned['text'] = cleaned['text'].apply(clean_text)
cleaned['text'] = cleaned['text'].str.replace('_',' ')

cleaned.head()

In [ ]:
def func(row):
    text = row['text']
    if row['emailcount'] > 0 or row['urlcount'] > 0:
        foundurl = row['url']
        foundemail = row['email']
        for each_url in foundurl:
            text = text.replace(each_url, ' ')
        for each_email in foundemail:
            text = text.replace(each_email, ' ')
    return text

cleaned['text_cleaned'] = cleaned.apply(func, axis=1)
cleaned.head()


In [ ]:
cleaned = cleaned.loc[:,['text_cleaned', 'class']]
cleaned.dropna()
cleaned

In [ ]:
sns.countplot(cleaned['class'])

# Sentiment Analysis with BERT

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

# load the pre-trained BERT Tokenizer and Sequence Classifier
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

# build our model with the Sequence Classifier and our tokenizer with BERT’s Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf

In [ ]:
cleaned.head()

In [ ]:
X = cleaned.text_cleaned
y = cleaned['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)